In [5]:
!wget https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar -O tabular-dl-tabr.tar.gz
!tar -xvf tabular-dl-tabr.tar.gz

--2024-11-16 12:26:06--  https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.118, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/1b/25/1b25d6e2556c827abfaf6ba9da6021338691cafc39fe9d77986955ae0a69243d/0d74e4b96febb48fbe4dd2d851f8638b5738354f82c3183f92adbd2abf2f256b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data.tar%3B+filename%3D%22data.tar%22%3B&response-content-type=application%2Fx-tar&Expires=1732019166&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjAxOTE2Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xYi8yNS8xYjI1ZDZlMjU1NmM4MjdhYmZhZjZiYTlkYTYwMjEzMzg2OTFjYWZjMzlmZTlkNzc5ODY5NTVhZTBhNjkyNDNkLzBkNzRlNGI5NmZlYmI0OGZiZTRkZDJkODUxZjg2MzhiNTczODM1NGY4MmMzMTgzZjkyYWRiZDJhYmYyZjI

In [1]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.3 MB/s eta 0:00:00


In [2]:
!pip install psutil

In [3]:
import pandas as pd
import numpy as np
#import optuna
import torch.optim
import psutil

from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import QuantileTransformer

In [ ]:
clf = TabNetClassifier()

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [6]:
X_train = np.load('/content/data/classif-num-medium-0-MagicTelescope/X_num_train.npy')
y_train = np.load('/content/data/classif-num-medium-0-MagicTelescope/Y_train.npy')

X_valid = np.load('/content/data/classif-num-medium-0-MagicTelescope/X_num_val.npy')
y_valid = np.load('/content/data/classif-num-medium-0-MagicTelescope/Y_val.npy')

X_test = np.load('/content/data/classif-num-medium-0-MagicTelescope/X_num_test.npy')
y_test = np.load('/content/data/classif-num-medium-0-MagicTelescope/Y_test.npy')

In [7]:
normalizer = QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X_train.shape[0] // 30, 1000), 10),
            subsample=1_000_000_000,
            )
normalizer.fit_transform(X_train)

array([[-1.6393101 , -0.90230376, -1.3945612 , ..., -0.29131973,
        -0.737393  , -0.467348  ],
       [ 0.54153097, -0.07611426, -1.0577202 , ..., -1.0765651 ,
        -0.4129752 , -0.04665878],
       [ 1.7132984 ,  1.0078336 ,  0.42975497, ..., -1.0916998 ,
         0.24872102,  2.1620865 ],
       ...,
       [-1.0245086 , -1.0647414 , -0.80412334, ..., -0.6278598 ,
         0.6391125 , -0.27882308],
       [-1.203026  , -1.1980292 , -1.1129854 , ...,  0.06195614,
         0.6075059 ,  0.02848465],
       [ 1.1269407 ,  1.5216641 ,  1.2471416 , ...,  2.1540494 ,
         0.318182  ,  2.4361293 ]], dtype=float32)

In [ ]:
#Baseline
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
)

epoch 0  | loss: 0.67768 | train_accuracy: 0.57332 | valid_accuracy: 0.56276 |  0:00:00s
epoch 1  | loss: 0.55612 | train_accuracy: 0.48425 | valid_accuracy: 0.49127 |  0:00:01s
epoch 2  | loss: 0.49823 | train_accuracy: 0.48339 | valid_accuracy: 0.48961 |  0:00:01s
epoch 3  | loss: 0.44585 | train_accuracy: 0.55121 | valid_accuracy: 0.5611  |  0:00:02s
epoch 4  | loss: 0.41527 | train_accuracy: 0.53508 | valid_accuracy: 0.55694 |  0:00:02s
epoch 5  | loss: 0.40777 | train_accuracy: 0.51287 | valid_accuracy: 0.53616 |  0:00:03s
epoch 6  | loss: 0.40038 | train_accuracy: 0.50304 | valid_accuracy: 0.52618 |  0:00:03s
epoch 7  | loss: 0.39284 | train_accuracy: 0.52035 | valid_accuracy: 0.54613 |  0:00:04s
epoch 8  | loss: 0.38707 | train_accuracy: 0.58763 | valid_accuracy: 0.61596 |  0:00:05s
epoch 9  | loss: 0.38277 | train_accuracy: 0.56713 | valid_accuracy: 0.58271 |  0:00:05s
epoch 10 | loss: 0.38625 | train_accuracy: 0.54235 | valid_accuracy: 0.57024 |  0:00:06s
epoch 11 | loss: 0.37

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
def objective(trial):
    params = {
        'n_d': trial.suggest_int('n_d', 8, 64),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        'momentum': trial.suggest_float('momentum', 0.01, 0.4),
    }

    model = TabNetClassifier(**params, n_a=params['n_d'])
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=['accuracy'], patience=10)

    # Evaluate model performance
    score = model.best_cost  # or any other metric

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params

[I 2024-07-06 05:35:34,206] A new study created in memory with name: no-name-d26137c2-df0a-4734-92d3-db32dc1ad167
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.98551 | val_0_accuracy: 0.51039 |  0:00:01s
epoch 1  | loss: 0.71362 | val_0_accuracy: 0.54281 |  0:00:02s
epoch 2  | loss: 0.65051 | val_0_accuracy: 0.5453  |  0:00:04s
epoch 3  | loss: 0.56729 | val_0_accuracy: 0.63259 |  0:00:05s
epoch 4  | loss: 0.51416 | val_0_accuracy: 0.57855 |  0:00:06s
epoch 5  | loss: 0.51703 | val_0_accuracy: 0.6251  |  0:00:08s
epoch 6  | loss: 0.53667 | val_0_accuracy: 0.665   |  0:00:09s
epoch 7  | loss: 0.51377 | val_0_accuracy: 0.60848 |  0:00:10s
epoch 8  | loss: 0.49653 | val_0_accuracy: 0.61845 |  0:00:12s
epoch 9  | loss: 0.48372 | val_0_accuracy: 0.63259 |  0:00:13s
epoch 10 | loss: 0.46441 | val_0_accuracy: 0.66168 |  0:00:14s
epoch 11 | loss: 0.461   | val_0_accuracy: 0.70324 |  0:00:16s
epoch 12 | loss: 0.46751 | val_0_accuracy: 0.70241 |  0:00:17s
epoch 13 | loss: 0.45615 | val_0_accuracy: 0.73982 |  0:00:18s
epoch 14 | loss: 0.43797 | val_0_accuracy: 0.75977 |  0:00:20s
epoch 15 | loss: 0.42962 | val_0_accuracy: 0.74065 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 05:37:00,251] Trial 0 finished with value: 0.8462177888611804 and parameters: {'n_d': 44, 'n_steps': 8, 'gamma': 1.3332993841099636, 'n_independent': 3, 'n_shared': 5, 'momentum': 0.05281675134578197}. Best is trial 0 with value: 0.8462177888611804.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.80572 | val_0_accuracy: 0.52785 |  0:00:00s
epoch 1  | loss: 0.88648 | val_0_accuracy: 0.51538 |  0:00:01s
epoch 2  | loss: 0.72946 | val_0_accuracy: 0.54281 |  0:00:01s
epoch 3  | loss: 0.58104 | val_0_accuracy: 0.53034 |  0:00:02s
epoch 4  | loss: 0.50115 | val_0_accuracy: 0.61845 |  0:00:03s
epoch 5  | loss: 0.50166 | val_0_accuracy: 0.62926 |  0:00:03s
epoch 6  | loss: 0.46865 | val_0_accuracy: 0.63924 |  0:00:04s
epoch 7  | loss: 0.45609 | val_0_accuracy: 0.70823 |  0:00:04s
epoch 8  | loss: 0.45225 | val_0_accuracy: 0.64422 |  0:00:05s
epoch 9  | loss: 0.50457 | val_0_accuracy: 0.73234 |  0:00:06s
epoch 10 | loss: 0.46709 | val_0_accuracy: 0.73649 |  0:00:06s
epoch 11 | loss: 0.43461 | val_0_accuracy: 0.74231 |  0:00:07s
epoch 12 | loss: 0.43463 | val_0_accuracy: 0.7448  |  0:00:07s
epoch 13 | loss: 0.44363 | val_0_accuracy: 0.71737 |  0:00:08s
epoch 14 | loss: 0.49088 | val_0_accuracy: 0.74979 |  0:00:09s
epoch 15 | loss: 0.48043 | val_0_accuracy: 0.7581  |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 05:37:32,287] Trial 1 finished with value: 0.8395677472984207 and parameters: {'n_d': 61, 'n_steps': 9, 'gamma': 1.992974766879843, 'n_independent': 1, 'n_shared': 1, 'momentum': 0.24181114885526883}. Best is trial 0 with value: 0.8462177888611804.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 06:47:09,010] Trial 97 finished with value: 0.85785536159601 and parameters: {'n_d': 50, 'n_steps': 3, 'gamma': 1.275098697558152, 'n_independent': 2, 'n_shared': 2, 'momentum': 0.0657935199341124}. Best is trial 42 with value: 0.8728179551122195.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.79065 | val_0_accuracy: 0.53533 |  0:00:00s
epoch 1  | loss: 0.54866 | val_0_accuracy: 0.56525 |  0:00:01s
epoch 2  | loss: 0.48886 | val_0_accuracy: 0.56692 |  0:00:01s
epoch 3  | loss: 0.46904 | val_0_accuracy: 0.58437 |  0:00:02s
epoch 4  | loss: 0.44355 | val_0_accuracy: 0.61679 |  0:00:03s
epoch 5  | loss: 0.42535 | val_0_accuracy: 0.60765 |  0:00:03s
epoch 6  | loss: 0.41856 | val_0_accuracy: 0.63924 |  0:00:04s
epoch 7  | loss: 0.41687 | val_0_accuracy: 0.67498 |  0:00:04s
epoch 8  | loss: 0.40435 | val_0_accuracy: 0.68246 |  0:00:05s
epoch 9  | loss: 0.39844 | val_0_accuracy: 0.6409  |  0:00:05s
epoch 10 | loss: 0.38984 | val_0_accuracy: 0.61264 |  0:00:06s
epoch 11 | loss: 0.38265 | val_0_accuracy: 0.58105 |  0:00:06s
epoch 12 | loss: 0.37807 | val_0_accuracy: 0.6143  |  0:00:07s
epoch 13 | loss: 0.37447 | val_0_accuracy: 0.68579 |  0:00:07s
epoch 14 | loss: 0.37997 | val_0_accuracy: 0.62178 |  0:00:08s
epoch 15 | loss: 0.37066 | val_0_accuracy: 0.63924 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 06:47:45,728] Trial 98 finished with value: 0.857024106400665 and parameters: {'n_d': 18, 'n_steps': 3, 'gamma': 1.3530861290893004, 'n_independent': 3, 'n_shared': 3, 'momentum': 0.11950769444590167}. Best is trial 42 with value: 0.8728179551122195.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.7555  | val_0_accuracy: 0.4522  |  0:00:00s
epoch 1  | loss: 0.52489 | val_0_accuracy: 0.52868 |  0:00:01s
epoch 2  | loss: 0.50154 | val_0_accuracy: 0.51704 |  0:00:01s
epoch 3  | loss: 0.4695  | val_0_accuracy: 0.5478  |  0:00:02s
epoch 4  | loss: 0.43392 | val_0_accuracy: 0.51787 |  0:00:03s
epoch 5  | loss: 0.41625 | val_0_accuracy: 0.51621 |  0:00:03s
epoch 6  | loss: 0.42293 | val_0_accuracy: 0.5212  |  0:00:04s
epoch 7  | loss: 0.42955 | val_0_accuracy: 0.51787 |  0:00:05s
epoch 8  | loss: 0.42552 | val_0_accuracy: 0.5212  |  0:00:05s
epoch 9  | loss: 0.40821 | val_0_accuracy: 0.53699 |  0:00:06s
epoch 10 | loss: 0.40304 | val_0_accuracy: 0.56027 |  0:00:07s
epoch 11 | loss: 0.39378 | val_0_accuracy: 0.57606 |  0:00:07s
epoch 12 | loss: 0.38768 | val_0_accuracy: 0.56775 |  0:00:08s
epoch 13 | loss: 0.38454 | val_0_accuracy: 0.55195 |  0:00:09s
epoch 14 | loss: 0.38752 | val_0_accuracy: 0.5877  |  0:00:10s
epoch 15 | loss: 0.38257 | val_0_accuracy: 0.68495 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 06:48:18,768] Trial 99 finished with value: 0.8470490440565254 and parameters: {'n_d': 37, 'n_steps': 4, 'gamma': 1.896765249264529, 'n_independent': 4, 'n_shared': 2, 'momentum': 0.1690083334652489}. Best is trial 42 with value: 0.8728179551122195.


In [ ]:
best_params

{'n_d': 30,
 'n_steps': 3,
 'gamma': 1.5673834303925407,
 'n_independent': 4,
 'n_shared': 2,
 'momentum': 0.10956480194746604}

In [8]:
best_params = {'n_d': 30,
 'n_steps': 3,
 'gamma': 1.5673834303925407,
 'n_independent': 4,
 'n_shared': 2,
 'momentum': 0.10956480194746604}

In [9]:
from pytorch_tabnet.callbacks import Callback

def print_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"System RAM Usage: {mem_info.rss / (1024 ** 2):.2f} MB")
    if torch.cuda.is_available():
      print(f"Allocated GPU memory: {torch.cuda.memory_allocated() / (1024 ** 2):.2f} MB")
      print(f"Cached GPU memory: {torch.cuda.memory_reserved() / (1024 ** 2):.2f} MB")

class MemoryUsageCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print_memory_usage()

In [ ]:
# Train final model using best hyperparameters
best_model = TabNetClassifier(
    **best_params,
    n_a=best_params['n_d'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2)
)

best_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    patience=16,
    callbacks=[MemoryUsageCallback()]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.64681 | train_accuracy: 0.5165  | valid_accuracy: 0.54364 |  0:00:02s
System RAM Usage: 1180.45 MB
Allocated GPU memory: 18.89 MB
Cached GPU memory: 60.00 MB
epoch 1  | loss: 0.47316 | train_accuracy: 0.55388 | valid_accuracy: 0.58105 |  0:00:03s
System RAM Usage: 1180.45 MB
Allocated GPU memory: 18.89 MB
Cached GPU memory: 60.00 MB
epoch 2  | loss: 0.44154 | train_accuracy: 0.49984 | valid_accuracy: 0.51787 |  0:00:04s
System RAM Usage: 1180.45 MB
Allocated GPU memory: 18.89 MB
Cached GPU memory: 60.00 MB
epoch 3  | loss: 0.41657 | train_accuracy: 0.49311 | valid_accuracy: 0.52203 |  0:00:05s
System RAM Usage: 1180.86 MB
Allocated GPU memory: 18.89 MB
Cached GPU memory: 60.00 MB
epoch 4  | loss: 0.41002 | train_accuracy: 0.50176 | valid_accuracy: 0.52535 |  0:00:05s
System RAM Usage: 1180.86 MB
Allocated GPU memory: 18.89 MB
Cached GPU memory: 60.00 MB
epoch 5  | loss: 0.39871 | train_accuracy: 0.50838 | valid_accuracy: 0.53034 |  0:00:06s
System RAM Usage: 1180.86 

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

Best Valid Score: 0.8703241895261845
Test AUC: 0.9170907466052919


In [ ]:
preds = best_model.predict(X_test)

print(classification_report(y_true=y_test, y_pred=preds))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84      1393
           1       0.85      0.83      0.84      1417

    accuracy                           0.84      2810
   macro avg       0.84      0.84      0.84      2810
weighted avg       0.84      0.84      0.84      2810



In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_score=y_proba[:,1], y_true=y_test)

roc_data = pd.DataFrame({
    'fpr': fpr,
    'tpr': tpr
})

roc_data.to_csv('MagicTelescope-TabNet.csv', index=False)